In [182]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
import os
import re

In [150]:
#subj_idx
subj_idxs=[]
for i in range(1,18):
    if len(str(i))==1:
        subj_idx='sub-00'+str(i)
        subj_idxs.append(subj_idx)
    else:
        subj_idx='sub-0'+str(i)
        subj_idxs.append(subj_idx)

['sub-001',
 'sub-002',
 'sub-003',
 'sub-004',
 'sub-005',
 'sub-006',
 'sub-007',
 'sub-008',
 'sub-009',
 'sub-010',
 'sub-011',
 'sub-012',
 'sub-013',
 'sub-014',
 'sub-015',
 'sub-016',
 'sub-017']

In [162]:
#tasks
tasks=[]
for side in ['outside','inside']:
    task='sourcedata-eeg_'+side+'-MRT'
    tasks.append(task)
tasks

['sourcedata-eeg_outside-MRT', 'sourcedata-eeg_inside-MRT']

In [225]:
#runs
runs=[]
for task in tasks:
    if task == tasks[0]:
        side = 'outside'
        for i in range(1,3):
            run = side+'MRT_run-0'+str(i)+'_beh.tsv'
            runs.append(run)
    else:
        side='inside'
        for i in range(1,6):
            run = side+'MRT_run-0'+str(i)+'_beh.tsv'
            runs.append(run)
runs     

['outsideMRT_run-01_beh.tsv',
 'outsideMRT_run-02_beh.tsv',
 'insideMRT_run-01_beh.tsv',
 'insideMRT_run-02_beh.tsv',
 'insideMRT_run-03_beh.tsv',
 'insideMRT_run-04_beh.tsv',
 'insideMRT_run-05_beh.tsv']

In [240]:
#df_dirs
df_dirs=[]
temp_dir =os.getcwd()
BIDS_data='BIDS_data'
datatype='beh'
for subj_idx in subj_idxs :
    for task in tasks:
        if task==tasks[0]:
            for run_index in range(0,2):
                run=runs[run_index]
                run=subj_idx+'_task-pdm_acq-'+run
                df_dir=os.path.join(temp_dir,BIDS_data,subj_idx,task,datatype,run)
                df_dirs.append(df_dir)
        else:
            for run_index in range(2,7):
                run=runs[run_index]
                run=subj_idx+'_task-pdm_acq-'+run
                df_dir=os.path.join(temp_dir,BIDS_data,subj_idx,task,datatype,run)
                df_dirs.append(df_dir)
df_dirs

['/home/jovyan/yikang1020/Ostwald_OSF/BIDS_data/sub-001/sourcedata-eeg_outside-MRT/beh/sub-001_task-pdm_acq-outsideMRT_run-01_beh.tsv',
 '/home/jovyan/yikang1020/Ostwald_OSF/BIDS_data/sub-001/sourcedata-eeg_outside-MRT/beh/sub-001_task-pdm_acq-outsideMRT_run-02_beh.tsv',
 '/home/jovyan/yikang1020/Ostwald_OSF/BIDS_data/sub-001/sourcedata-eeg_inside-MRT/beh/sub-001_task-pdm_acq-insideMRT_run-01_beh.tsv',
 '/home/jovyan/yikang1020/Ostwald_OSF/BIDS_data/sub-001/sourcedata-eeg_inside-MRT/beh/sub-001_task-pdm_acq-insideMRT_run-02_beh.tsv',
 '/home/jovyan/yikang1020/Ostwald_OSF/BIDS_data/sub-001/sourcedata-eeg_inside-MRT/beh/sub-001_task-pdm_acq-insideMRT_run-03_beh.tsv',
 '/home/jovyan/yikang1020/Ostwald_OSF/BIDS_data/sub-001/sourcedata-eeg_inside-MRT/beh/sub-001_task-pdm_acq-insideMRT_run-04_beh.tsv',
 '/home/jovyan/yikang1020/Ostwald_OSF/BIDS_data/sub-001/sourcedata-eeg_inside-MRT/beh/sub-001_task-pdm_acq-insideMRT_run-05_beh.tsv',
 '/home/jovyan/yikang1020/Ostwald_OSF/BIDS_data/sub-002/so

In [567]:
#dfs
dfs=pd.DataFrame()
for df_dir in df_dirs:
    if os.path.exists(df_dir):
        df=pd.read_csv(df_dir,sep='\t')
        subject=''.join(re.findall(r'BIDS_data/(.+?)/sourcedata',df_dir))
        side=''.join(re.findall(r'/sourcedata-eeg_(.+?)-MRT/beh/',df_dir))
        run=''.join(re.findall(r'run-0(.+?)_beh',df_dir))
        df['subject']=subject
        df['side']=side
        df['run']=int(run)
        dfs=pd.concat([df,dfs])
    else:
        pass

In [568]:
#clear
dfs=dfs.reset_index(inplace=False)
dfs=dfs.rename(columns={'level_0':'new_index'})
dfs['subject']=dfs['subject'].str.replace('sub-0',' ').astype(int)
dfs.dropna(subset=['response_time'],inplace=True)

#variable
dfs['prioritization_cue']=dfs['prioritization_cue'].map({74:'left',75:'right',76:'double'},na_action=None)
dfs['coherency']=dfs['condition'].map({1:'high',2:'high',3:'low',4:'low'})
dfs['prioritization']=dfs['condition'].map({1:'yes',2:'no',3:'yes',4:'no'})

car_images=[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54]
face_images=[19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72]
dfs['category']=dfs['image_index'].isin(car_images).astype(int).map({1:'car', 0:'face'})

dfs.to_csv('/home/jovyan/yikang1020/Ostwald_OSF/dfs_raw')